## Deliverable 3. Create a Travel Itinerary Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [13]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df=vacation_df.rename(columns={"Current_Description":"Current Weather"})
clean_vacation_df=vacation_df.dropna()
clean_vacation_df

,City_ID,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,5,Cape Town,ZA,61.27,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
2,7,Nemuro,JP,61.23,broken clouds,43.3236,145.5750,Hotel Nemuro Kaiyoutei
3,9,Margate,GB,61.30,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
4,10,Jinchang,CN,61.65,scattered clouds,38.4953,102.1739,Mingshihui Jinchang International Hotel
5,15,Airai,TL,59.74,broken clouds,-8.9266,125.4092,Juvinal Martin
...,...,...,...,...,...,...,...,...
190,661,Portsmouth,GB,61.00,overcast clouds,50.7990,-1.0913,Best Western Royal Beach Hotel
191,666,Imbituba,BR,65.50,overcast clouds,-28.2400,-48.6703,Praia Hotel Imbituba
192,672,Simeonovgrad,BG,56.32,overcast clouds,42.0333,25.8333,ПРЕВОЭ ТАКСИТА
193,674,Boo,SE,53.76,overcast clouds,59.3333,18.2833,Yasuragi


In [14]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd> and <dd>{Max Temp} F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_vacation_df[["Lat", "Lng"]]

In [15]:
# 4a. Add a marker layer for each city to the map.
locations = clean_vacation_df[["Lat", "Lng"]]
max_temp = clean_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [19]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = clean_vacation_df.loc[clean_vacation_df["City"]=="Thompson"]
vacation_end = clean_vacation_df.loc[clean_vacation_df["City"]=="Thompson"]
vacation_stop1 = clean_vacation_df.loc[clean_vacation_df["City"]=="La Ronge"]
vacation_stop2 = clean_vacation_df.loc[clean_vacation_df["City"]=="High Prairie"] 
vacation_stop3 = clean_vacation_df.loc[clean_vacation_df["City"]=="Yellowknife"] 
print(vacation_start)

    City_ID      City Country  Max Temp Current Weather      Lat      Lng  \
43      157  Thompson      CA     65.93      few clouds  55.7435 -97.8558   

         Hotel Name  
43  Burntwood Hotel  


In [17]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3= vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')

Start: (55.7435, -97.8558), End: (55.7435, -97.8558)
Stop 1: (55.1001, -105.2842), Stop 2: (55.4334, -116.4858), Stop 3: (62.456, -114.3525)


In [26]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='BICYCLING')

fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [27]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,157,Thompson,CA,65.93,few clouds,55.7435,-97.8558,Burntwood Hotel
1,157,Thompson,CA,65.93,few clouds,55.7435,-97.8558,Burntwood Hotel
2,560,La Ronge,CA,64.80,broken clouds,55.1001,-105.2842,Northland Motor Hotel
3,390,High Prairie,CA,64.78,overcast clouds,55.4334,-116.4858,Royal Star Motor Inn
4,44,Yellowknife,CA,51.42,overcast clouds,62.4560,-114.3525,The Explorer Hotel


In [29]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd> and <dd>{Max Temp} F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map.
max_temp = clean_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))